In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
metadata = pd.read_csv('./movies_metadata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [ ]:
metadata.shape

(45466, 24)

In [ ]:
keywords = pd.read_csv('./keywords.csv')

In [ ]:
keywords.head(2)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [ ]:
keywords.shape

(46419, 2)

In [ ]:
metadata = metadata[metadata['id'] != '1997-08-20']
metadata = metadata[metadata['id'] != '2012-09-20']
metadata = metadata[metadata['id'] != '2012-09-29']
metadata = metadata[metadata['id'] != '2014-01-01']

In [ ]:
metadata = metadata.astype({'id':int})
keywords = keywords.astype({'id':int})

data = metadata.merge(keywords, on='id', how='left')

In [ ]:
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [ ]:
data = data[['id','genres', 'vote_average', 'vote_count','popularity','title',  'keywords', 'overview']]

In [ ]:
m = data['vote_count'].quantile(0.99)
m

2153.9800000000105

In [ ]:
data = data.copy().loc[data['vote_count'] >= m]
data.shape

(465, 8)

In [ ]:
data.head(2)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,21.9469,Toy Story,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","Led by Woody, Andy's toys live happily in his ..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,17.0155,Jumanji,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",When siblings Judy and Peter discover an encha...


In [ ]:
C = data['vote_average'].mean()
C

6.990322580645161

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return ( v / (v+m) * R ) + (m / (m + v) * C)

In [ ]:
data['score'] = data.apply(weighted_rating, axis = 1)

In [ ]:
data.head(2)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",7.7,5415.0,21.9469,Toy Story,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","Led by Woody, Andy's toys live happily in his ...",7.49804
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",6.9,2413.0,17.0155,Jumanji,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",When siblings Judy and Peter discover an encha...,6.94260


In [ ]:
data.shape

(465, 9)

In [ ]:
data['genres'] = data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)

In [ ]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [ ]:
data['genres'] = data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))
data['keywords'] = data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))


In [ ]:
data.head(2)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,862,Animation Comedy Family,7.7,5415.0,21.9469,Toy Story,jealousy toy boy friendship friends rivalry bo...,"Led by Woody, Andy's toys live happily in his ...",7.49804
1,8844,Adventure Fantasy Family,6.9,2413.0,17.0155,Jumanji,board game disappearance based on children's b...,When siblings Judy and Peter discover an encha...,6.94260


In [ ]:
data.to_csv('./pre_tmdb_5000_movies.csv', index = False)

In [ ]:
count_vector = CountVectorizer(ngram_range=(1, 3))

In [ ]:
c_vector_genres = count_vector.fit_transform(data['genres'])

In [ ]:
c_vector_genres.shape

(465, 355)

In [ ]:
#코사인 유사도를 구한 벡터를 미리 저장
gerne_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:,::-1]

In [ ]:
gerne_c_sim.shape

(465, 465)

In [ ]:
print(gerne_c_sim) # cosine_similarity(arg1, arg2)

[[1.         0.16666667 0.         ... 0.         0.         0.        ]
 [0.16666667 1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.66666667 0.13608276 0.09622504]
 ...
 [0.         0.         0.66666667 ... 1.         0.13608276 0.09622504]
 [0.         0.         0.13608276 ... 0.13608276 1.         0.23570226]
 [0.         0.         0.09622504 ... 0.09622504 0.23570226 1.        ]]


In [ ]:
print(gerne_c_sim) #.argsort()

[[232 292 291 ... 168 313   0]
 [232 275 266 ...   1 123 102]
 [  0 318 317 ... 400 365   2]
 ...
 [  0 149 150 ... 319 311 462]
 [  0 136 326 ... 118 278 463]
 [  0 119 265 ... 228   5 464]]


In [ ]:
print(gerne_c_sim) #.argsort()[:]

[[232 292 291 ... 168 313   0]
 [232 275 266 ...   1 123 102]
 [  0 318 317 ... 400 365   2]
 ...
 [  0 149 150 ... 319 311 462]
 [  0 136 326 ... 118 278 463]
 [  0 119 265 ... 228   5 464]]


In [ ]:
print(gerne_c_sim) #.argsort()[:, ::-1]

[[  0 313 168 ... 291 292 232]
 [102 123   1 ... 266 275 232]
 [  2 365 400 ... 317 318   0]
 ...
 [462 311 319 ... 150 149   0]
 [463 278 118 ... 326 136   0]
 [464   5 228 ... 265 119   0]]


In [ ]:
print(gerne_c_sim) #.argsort()[:, :-1]

[[232 292 291 ... 459 168 313]
 [232 275 266 ...  90   1 123]
 [  0 318 317 ... 231 400 365]
 ...
 [  0 149 150 ...   2 319 311]
 [  0 136 326 ...  87 118 278]
 [  0 119 265 ... 323 228   5]]


In [ ]:
def get_recommend_movie_list(df, movie_title, top=30):
    # 특정 영화와 비슷한 영화를 추천해야 하기 때문에 '특정 영화' 정보를 뽑아낸다.
    target_movie_index = df[df['title'] == movie_title].index.values
    print("target_movie_index")
    print(target_movie_index)
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = gerne_c_sim[target_movie_index, :top].reshape(-1)
    print("sim_index")
    print(sim_index)
    print(sim_index.shape)
    #본인을 제외
    sim_index = sim_index[sim_index != target_movie_index]
    print("sim_index without target")
    print(sim_index)
    print(sim_index.shape)

    #data frame으로 만들고 vote_count으로 정렬한 뒤 return
    result = df.iloc[sim_index].sort_values('score', ascending=False)[:10]
    return result

In [ ]:
get_recommend_movie_list(data, movie_title='Toy Story')

target_movie_index
[0]
sim_index
[  0 313 168 459  75  89 279 159  65 196 433 199  95 163 336  17  85 121
 296 356 282 272 233 148 216 309 108 229 410 181]
(30,)
sim_index without target
[313 168 459  75  89 279 159  65 196 433 199  95 163 336  17  85 121 296
 356 282 272 233 148 216 309 108 229 410 181]
(29,)


,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
13751,14160,Animation Comedy Family Adventure,7.8,7048.0,19.3309,Up,age difference central and south america ballo...,Carl Fredricksen spent his entire life dreamin...,7.610472
12724,10681,Animation Family,7.8,6439.0,16.0884,WALL·E,romantic comedy,WALL·E is the last robot left on an Earth that...,7.597040
6243,12,Animation Family,7.6,6292.0,25.4978,Finding Nemo,father son relationship harbor underwater fish...,"Nemo, an adventurous young clownfish, is unexp...",7.444514
15379,10193,Animation Family Comedy,7.6,4710.0,16.9665,Toy Story 3,hostage college toy barbie animation escape da...,"Woody, Buzz, and the rest of Andy's toys haven...",7.408678
4763,585,Animation Comedy Family,7.5,6150.0,26.42,"Monsters, Inc.",monster infant energy supply company rivalry h...,"James Sullivan and Mike Wazowski are monsters,...",7.367794
23603,82702,Fantasy Action Adventure Animation Comedy Family,7.6,3163.0,12.2567,How to Train Your Dragon 2,father son relationship wife husband relations...,The thrilling second chapter of the epic How T...,7.353011
11586,2062,Animation Comedy Family Fantasy,7.5,4510.0,20.508,Ratatouille,paris brother brother relationship expensive r...,A rat named Remy dreams of becoming a great Fr...,7.335258
22762,137106,Adventure Animation Comedy Family Fantasy,7.5,3127.0,16.4181,The Lego Movie,father son relationship creativity friendship ...,"An ordinary Lego mini-figure, mistakenly thoug...",7.292115
16163,38757,Animation Family,7.4,3419.0,14.6848,Tangled,hostage magic horse fairy tale musical princes...,When the kingdom's most wanted-and most charmi...,7.241658
4184,808,Adventure Animation Comedy Family Fantasy,7.3,4183.0,17.9877,Shrek,magic liberation lordship castle robin hood en...,It ain't easy bein' green -- especially if you...,7.194739


In [ ]:
data[data['title'] == 'Toy Story']

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,862,Animation Comedy Family,7.7,5415.0,21.9469,Toy Story,jealousy toy boy friendship friends rivalry bo...,"Led by Woody, Andy's toys live happily in his ...",7.49804
